**Tujuan** dari program ini adalah melakukan crawling (pengambilan) data komentar pada sebuah video Youtube menggunakan **Youtube Data API v3**. Sebelum mencoba program ini, pastikan Anda sudah memiliki (mengaktifkan) layanan Youtube Data API dan telah membangkitkan **API Key**. 

Jika belum memiliki **API KEY**, Anda dapat mengikuti petunjuk singkat sebagai berikut: 
1. Login ke Google Developer Console (https://console.developers.google.com/)dengan akun Google Anda
2. Buat project baru dan lengkapi isian yang diminta. 
3. Aktifkan Layanan API pada halaman project, dan cari **Youtube Data API v3**.
4. Dari halaman dashboard, buat kredential agar API tersebut dapat digunakan. Klik tombol **Buat Kredensial** (**Create Credential**). Lengkapi isian formnya.
5. Anda dapat mengakses / melihat API KEY pada tab **Credentials**.



#1. Import Library

In [1]:
import pandas as pd
from googleapiclient.discovery import build

#2. Fungsi untuk crawling komentar

In [2]:
def video_comments(video_id):
	# empty list for storing reply
	replies = []

	# creating youtube resource object
	youtube = build('youtube', 'v3', developerKey=api_key)

	# retrieve youtube video results
	video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

	# iterate video response
	while video_response:
		
		# extracting required info
		# from each result object
		for item in video_response['items']:
			
			# Extracting comments ()
			published = item['snippet']['topLevelComment']['snippet']['publishedAt']
			user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']

			# Extracting comments
			comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
			likeCount = item['snippet']['topLevelComment']['snippet']['likeCount']

			replies.append([published, user, comment, likeCount])
			
			# counting number of reply of comment
			replycount = item['snippet']['totalReplyCount']

			# if reply is there
			if replycount>0:
				# iterate through all reply
				for reply in item['replies']['comments']:
					
					# Extract reply
					published = reply['snippet']['publishedAt']
					user = reply['snippet']['authorDisplayName']
					repl = reply['snippet']['textDisplay']
					likeCount = reply['snippet']['likeCount']
					
					# Store reply is list
					#replies.append(reply)
					replies.append([published, user, repl, likeCount])

			# print comment with list of reply
			#print(comment, replies, end = '\n\n')

			# empty reply list
			#replies = []

		# Again repeat
		if 'nextPageToken' in video_response:
			video_response = youtube.commentThreads().list(
					part = 'snippet,replies',
					pageToken = video_response['nextPageToken'], 
					videoId = video_id
				).execute()
		else:
			break
	#endwhile
	return replies


#3. Jalankan Proses Crawling

In [ ]:
# isikan dengan api key Anda
api_key = '<<API KEY>>'

# Enter video id
# contoh url video = https://www.youtube.com/watch?v=5tucmKjOGi8
video_id = "5tucmKjOGi8" #isikan dengan kode / ID video

# Call function
comments = video_comments(video_id)

comments

#4. Ubah Hasil Crawling ke Dataframe

In [ ]:
df = pd.DataFrame(comments, columns=['publishedAt', 'authorDisplayName', 'textDisplay', 'likeCount'])
df

#5. Simpan Hasil Crawling ke file CSV

In [5]:
df.to_csv('youtube-comments.csv', index=False)